# <font color=yellow> **Modelo sísmico**

In [117]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import resample
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [118]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [122]:
pip install -U kaleido

## <font color=yellow> **Criando o modelo sísmico**


In [119]:
# Condições iniciais do reservatório
Por = 0.35      # Porosidade
K_min = 38      # Módulo de volume da rocha (GPa)
mu_min = 44     # Módulo de cisalhamento da rocha (GPa)
Rho_min = 2.65  # Densidade do quartzo (g/cm^3)

# Condições iniciais dos fluidos do reservatório
filtrate_sat = np.array([0.0, 0.05, 0.2, 0.4, 0.6, 1.0])  # Fração de saturação do filtrado (água)
K_hc = 1.15   # Módulo de volume do óleo em GPa
K_filtrate = 3.0  # Módulo de volume da água em GPa
rho_hc = 0.8    # Densidade do óleo em g/cm^3
rho_mud = 1.4   # Densidade da água em g/cm^3

K_fluid_initial = K_hc

# Calculando a densidade inicial da camada c3
rho_c3_initial = Rho_min * (1 - Por) + Por * rho_hc
print("Densidade inicial da camada c3:", rho_c3_initial)

# # Calculando o módulo de volume inicial
# K_bulk_initial = ((K_min * (1 - Por) + K_fluid_initial * Por) +1/((1 - Por)/K_min + Por/ K_fluid_initial))/2
# mu_bulk_initial = mu_min * (1 - Por)

# # Calculando a velocidade de Vs da camada c3
# Vs_c3 = np.sqrt(mu_bulk_initial / rho_c3_initial)

# # Calculando a velocidade de Vp da camada c3
# Vp_c3 = np.sqrt((K_bulk_initial + (4 / 3) * mu_bulk_initial) / rho_c3_initial)

# print("Velocidade de Vs da camada c3:", Vs_c3)
# print("Velocidade de Vp da camada c3:", Vp_c3)


Densidade inicial da camada c3: 2.0025


In [120]:
# depth axis
z = np.arange(0, 1000, 1)
n_z = len(z)

# time axis
t = np.arange(0, 4.0, 0.004)
n_t = len(t)

# CMP
cmp = np.arange(1, 11)
n_cmp = len(cmp)

# reservoir thikness
reservoir = 570

# vp values for each layer
vp_c1 = 2.8 # sandstone
vp_c2 = 4.4
vp_c3 = 2.5 # high porosity sandstone (reservoir)
vp_c4 = 5.0
vp_c5 = 6.5 # limestone

# vs values for each layer
vs_c1 = 1.5 # sandstone
vs_c2 = 2.2
vs_c3 = 1.5 # high porosity sandstone (reservoir)
vs_c4 = 3.5
vs_c5 = 4.20 # limestone


# rho values for each layer
rho_c1 = 2.1 # sandstone
rho_c2 = 2.4
rho_c3 = rho_c3_initial # high porosity sandstone (reservoir)
rho_c4 = 2.6
rho_c5 = 2.7 # limestone

# vp model
vp = np.zeros((n_z, n_cmp))
vp[0:250, :] = vp_c1
vp[250:500, :] = vp_c2
vp[500:reservoir, :] = vp_c3
vp[reservoir:750, :] = vp_c4
vp[750:1000, :] = vp_c5

for i in range(n_cmp):
    vp[0:250, i] = vp_c1 + (i/1000)
    vp[250:500, i] = vp_c2 + 5*(i/1000)
    vp[500:reservoir, i] = vp_c3 + 10*(i/1000)
    vp[reservoir:750, i] = vp_c4 + (i/1000)
    vp[750:1000, i] = vp_c5 + 10*(i/1000)

# vs model
vs = np.zeros((n_z, n_cmp))
vs[0:250, :] = vs_c1
vs[250:500, :] = vs_c2
vs[500:reservoir, :] = vs_c3
vs[reservoir:750, :] = vs_c4
vs[750:1000, :] = vs_c5

for i in range(n_cmp):
    vs[0:100, i] = vs_c1 + (i/1000)
    vs[250:500, i] = vs_c2 + 5*(i/1000)
    vs[500:reservoir, i] = vs_c3 + 10*(i/1000)
    vs[reservoir:750, i] = vs_c4 + (i/1000)
    vs[750:1000, i] = vs_c5 + 10*(i/1000)

# rho model
rho = np.zeros((n_z, n_cmp))
rho[0:250, :] = rho_c1
rho[250:500, :] = rho_c2
rho[500:reservoir, :] = rho_c3
rho[reservoir:750, :] = rho_c4
rho[750:1000, :] = rho_c5

for i in range(n_cmp):
    rho[0:250, i] = rho_c1 + (i/1000)
    rho[250:500, i] = rho_c2 + 5*(i/1000)
    rho[500:reservoir, i] = rho_c3 + 10*(i/1000)
    rho[reservoir:750, i] = rho_c4 + (i/1000)
    rho[750:1000, i] = rho_c5 + 10*(i/1000)

In [123]:
# vp model plot
fig = go.Figure()

fig.add_trace(go.Heatmap(z=vp[::-1], zmin=vp.min(), zmax=vp.max(), colorscale='Jet',
                         x=[str(c) for c in cmp], y=[str(int(d)) for d in z[::-1]]))

fig.update_layout(
    title="<b>Vp model (km/s)<b>",
    title_x=0.5,
    xaxis=dict(title="CMP"),
    yaxis=dict(title="Depth (m)"),
    coloraxis=dict(colorbar=dict(title="Compressional velocity (m/s)", len=0.5, y=0.5)),
    height=500, width=500,
    font=dict(size=18)
)

fig.update_yaxes(tickvals=list(range(0, 1001, 100)))

fig.show()

fig.write_image("/content/drive/MyDrive/doutorado/mud_filtrate/figures/fig_vp_model.png")

ValueError: 
Image export using the "kaleido" engine requires the kaleido package,
which can be installed using pip:
    $ pip install -U kaleido


In [ ]:
# vs model plot
fig = go.Figure()

fig.add_trace(go.Heatmap(z=vs[::-1], zmin=vs.min(), zmax=vs.max(), colorscale='Jet',
                         x=[str(c) for c in cmp], y=[str(int(d)) for d in z[::-1]]))

fig.update_layout(
    title="<b>Vs model (km/s)<b>",
    title_x=0.5,
    xaxis=dict(title="CMP"),
    yaxis=dict(title="Depth (m)"),
    coloraxis=dict(colorbar=dict(title="Compressional velocity (m/s)", len=0.5, y=0.5)),
    height=500, width=500,
    font=dict(size=18)
)

fig.update_yaxes(tickvals=list(range(0, 1001, 100)))

fig.show()

In [ ]:
# rho model plot
fig = go.Figure()

fig.add_trace(go.Heatmap(z=rho[::-1], zmin=rho.min(), zmax=rho.max(), colorscale='Jet',
                         x=[str(c) for c in cmp], y=[str(int(d)) for d in z[::-1]]))

fig.update_layout(
    title="<b>Rho model (g/cm³)<b>",
    title_x=0.5,
    xaxis=dict(title="CMP"),
    yaxis=dict(title="Depth (m)"),
    coloraxis=dict(colorbar=dict(title="Density (g/cm³)", len=0.5, y=0.5)),
    height=500, width=500,
    font=dict(size=18)
)

fig.update_yaxes(tickvals=list(range(0, 901, 100)))

fig.show()

In [ ]:
# vp and rho plot
fig = make_subplots(rows=1, cols=3, subplot_titles=('Vp', 'Rho', 'Vs'))

fig.add_trace(go.Scatter(x=vp[:,0], mode='lines', name='Seismic trace',
                         line=dict(color='darkblue')), row=1, col=1)
fig.add_trace(go.Scatter(x=rho[:,0], mode='lines', name='Seismic trace',
                         line=dict(color='crimson')), row=1, col=2)
fig.add_trace(go.Scatter(x=vs[:,0], mode='lines', name='Seismic trace',
                         line=dict(color='crimson')), row=1, col=3)

# xaxis properties
fig.update_xaxes(title_text='(km/s)', row=1, col=1,
                 title_font=dict(size=22), tickfont=dict(size=20))
fig.update_xaxes(title_text='(g/cm³)', row=1, col=2,
                 title_font=dict(size=22), tickfont=dict(size=20))
fig.update_xaxes(title_text='(km/s)', row=1, col=3,
                 title_font=dict(size=22), tickfont=dict(size=20))

# yaxis properties
fig.update_yaxes(title_text='Depth (m)', row=1, col=1, autorange="reversed",
                 title_font=dict(size=22), tickfont=dict(size=20))
fig.update_yaxes(row=1, col=2, autorange="reversed",
                 title_font=dict(size=22), tickfont=dict(size=20))
fig.update_yaxes(row=1, col=3, autorange="reversed",
                 title_font=dict(size=22), tickfont=dict(size=20))

fig.update_layout(height=500, width=900, showlegend=False, title_text='Vp, Vs and Rho seismic model',
                  title_font=dict(size=24), title_x=0.5)

fig.update_annotations(font=dict(size=24))

fig.show()

## <font color=yellow> **Gerando o traço sintético**

In [ ]:
# selecting CMP
vp_trace = vp[:,0]
vs_trace = vs[:,0]
rho_trace = rho[:,0]

# wavelet Ricker
f = 20  # Hz
w = np.zeros(len(t))
for i in range(len(t)):
    w[i] = (1 - (2 * np.pi ** 2 * f ** 2 * (t[i] - 0.08) ** 2)) * np.exp(-np.pi ** 2 * f ** 2 * (t[i] - 0.08) ** 2)

# impedance
ip = np.zeros((n_z))
ip = vp_trace * rho_trace

# reflectivity
R = np.zeros((n_z))
for i in range(n_z - 1):
  R[i] = (ip[i + 1] - ip[i]) / (ip[i + 1] + ip[i])

# Convolution
synt = np.zeros(n_z)
synt = np.convolve(R, w)

# creating a seismic trace with noise
synt_r = np.zeros(n_z)
for i in range(n_z):
    h = np.random.rand(1) / 25
    synt_r[i] = synt[i] + h

In [ ]:
# synthetic seismic trace plot
fig = make_subplots(rows=1, cols=4, subplot_titles=('Impedance', 'Reflectivity', 'Wavelet', 'Seismic trace'))

fig.add_trace(go.Scatter(x=ip, y=t, name='Impedance',
                         line=dict(color='blue')), row=1, col=1)
fig.add_trace(go.Scatter(x=R, y=t, name='Reflectivity',
                         line=dict(color='blue')), row=1, col=2)
fig.add_trace(go.Scatter(x=w, y=t, name='Wavelet',
                         line=dict(color='blue')), row=1, col=3)
fig.add_trace(go.Scatter(x=synt_r, y=t, name='Seismic trace',
                         line=dict(color='blue')), row=1, col=4)

# xaxis properties
fig.update_xaxes(title_text='kg/(m²s)', row=1, col=1,
                 title_font=dict(size=22), tickfont=dict(size=20))
fig.update_xaxes(title_text='Unitless', row=1, col=2,
                 title_font=dict(size=22), tickfont=dict(size=20))
fig.update_xaxes(title_text='Amplitude (a.u.)', row=1, col=3,
                 title_font=dict(size=22), tickfont=dict(size=20))
fig.update_xaxes(title_text='Amplitude (a.u.)', row=1, col=4,
                 title_font=dict(size=22), tickfont=dict(size=20))

# yaxis properties
fig.update_yaxes(title_text="Time (s)", row=1, col=1, autorange="reversed",
                 title_font=dict(size=22), tickfont=dict(size=20))
fig.update_yaxes(autorange="reversed", row=1, col=2,
                 title_font=dict(size=22), tickfont=dict(size=20))
fig.update_yaxes(autorange="reversed", range=[0, 3], row=1, col=3,
                 title_font=dict(size=22), tickfont=dict(size=20))
fig.update_yaxes(autorange="reversed", row=1, col=4,
                 title_font=dict(size=22), tickfont=dict(size=20))

fig.update_layout(height=600, width=1200, showlegend=False,
                  title_text=f'Seismic model: thickness: {len(vp[500:reservoir])}m, porosity: {round(Por*100)}%, frequency: {f} Hz ',
                  title_font=dict(size=24), title_x=0.5)

fig.update_annotations(font=dict(size=24))

fig.show()

In [ ]:
np.savetxt(f'/content/drive/MyDrive/doutorado/mud_filtrate/dados/initial_trace_{len(vp[500:reservoir])}_{round(Por*100)}_{f}.txt', synt_r)
np.savetxt(f'/content/drive/MyDrive/doutorado/mud_filtrate/dados/initial_wavelet_{f}.txt', w)

In [ ]:
# Define uma função para adicionar margem nos eixos
def add_margin(data, margin_ratio=0.15):
    min_value = min(data)
    max_value = max(data)
    range_margin = (max_value - min_value) * margin_ratio
    return [min_value - range_margin, max_value + range_margin]

In [ ]:
# synthetic seismic trace plot
fig = make_subplots(rows=1, cols=6, subplot_titles=('Velocity', 'Density', 'Impedance', 'Reflectivity', 'Wavelet', 'Seismic trace'), horizontal_spacing=0.03)

fig.add_trace(go.Scatter(x=vp_trace, y=t, name='Vp',
                         line=dict(color='#2E91E5')), row=1, col=1)
fig.add_trace(go.Scatter(x=rho_trace, y=t, name='Rho',
                         line=dict(color='#AF0038')), row=1, col=2)
fig.add_trace(go.Scatter(x=ip, y=t, name='Impedance',
                         line=dict(color='#222A2A')), row=1, col=3)
fig.add_trace(go.Scatter(x=R, y=t, name='Reflectivity',
                         line=dict(color='#222A2A')), row=1, col=4)
fig.add_trace(go.Scatter(x=w, y=t, name='Wavelet',
                         line=dict(color='#222A2A')), row=1, col=5)
fig.add_trace(go.Scatter(x=synt_r, y=t, name='Seismic trace',
                         line=dict(color='#222A2A')), row=1, col=6)

# xaxis properties
fig.update_xaxes(title_text='km/s', row=1, col=1,
                 title_font=dict(size=22), tickfont=dict(size=20),
                 range=add_margin(vp_trace))
fig.update_xaxes(title_text='g/cm³', row=1, col=2,
                 title_font=dict(size=22), tickfont=dict(size=20),
                 range=add_margin(rho_trace))
fig.update_xaxes(title_text='kg/(m²s)', row=1, col=3,
                 title_font=dict(size=22), tickfont=dict(size=20),
                 range=add_margin(ip))
fig.update_xaxes(title_text='Unitless', row=1, col=4,
                 title_font=dict(size=22), tickfont=dict(size=20),
                 range=add_margin(R))
fig.update_xaxes(title_text='(a.u.)', row=1, col=5,
                 title_font=dict(size=22), tickfont=dict(size=20),
                 range=add_margin(w))
fig.update_xaxes(title_text='(a.u.)', row=1, col=6,
                 title_font=dict(size=22), tickfont=dict(size=20),
                 range=add_margin(synt_r))

# yaxis properties
fig.update_yaxes(title_text="Time (s)", row=1, col=1, autorange="reversed",
                 title_font=dict(size=22), tickfont=dict(size=20))
fig.update_yaxes(autorange="reversed", row=1, col=2,
                 showticklabels=False)
fig.update_yaxes(autorange="reversed", row=1, col=3,
                 showticklabels=False)
fig.update_yaxes(autorange="reversed", row=1, col=4,
                 showticklabels=False)
fig.update_yaxes(autorange="reversed", row=1, col=5,
                 showticklabels=False)
fig.update_yaxes(autorange="reversed", row=1, col=6,
                 showticklabels=False)

# adds the letters
fig.add_annotation(x=min(vp_trace), y=4, xref='paper', yref='paper',
                   text=f'<b>(a)<b>', showarrow=False, row=1, col=1, font=dict(size=20))
fig.add_annotation(x=min(rho_trace), y=4, xref='paper', yref='paper',
                   text=f'<b>(b)<b>', showarrow=False, row=1, col=2, font=dict(size=20))
fig.add_annotation(x=min(ip), y=4, xref='paper', yref='paper',
                   text=f'<b>(c)<b>', showarrow=False, row=1, col=3, font=dict(size=20))
fig.add_annotation(x=min(R), y=4, xref='paper', yref='paper',
                   text=f'<b>(d)<b>', showarrow=False, row=1, col=4, font=dict(size=20))
fig.add_annotation(x=min(w), y=4, xref='paper', yref='paper',
                   text=f'<b>(e)<b>', showarrow=False, row=1, col=5, font=dict(size=20))
fig.add_annotation(x=min(synt_r), y=4, xref='paper', yref='paper',
                   text=f'<b>(f)<b>', showarrow=False, row=1, col=6, font=dict(size=20))

fig.update_layout(height=500, width=1300, showlegend=False,
                  title_text=f'<b>Seismic model: thickness: {len(vp[500:reservoir])}m, porosity: {round(Por*100)}%, frequency: {f} Hz<b>',
                  title_font=dict(size=24), title_x=0.5)

fig.update_annotations(font=dict(size=24))


fig.show()